In [1]:
import pandas as pd

In [2]:
import requests
domain = "ajc.com"

def get_pagerank(domain):
    headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
    request = requests.get(url, headers=headers)
    if request.status_code == 200:
        result = request.json()
        if result['status_code'] == 200:
            pr_int = result['response'][0]['page_rank_integer']
            pr_dec = result['response'][0]['page_rank_decimal']
            rank = result['response'][0]['rank']

            return pr_int, pr_dec, rank



In [3]:

import whois

def whois_checker(domain):
    w = whois.whois(domain)
    domain_registrar = w.get('registrar')
    postal_code = w.get('registrant_postal_code')
    country = w.get('country')

    return  domain_registrar, postal_code, country



In [4]:
import virustotal_python
from pprint import pprint
from base64 import urlsafe_b64encode
import json


def virus_scan(domain):
    with virustotal_python.Virustotal("242305dedab935c6a39736d0749ae88f2b68f1e2f0c8e82c81d8955c5fd194d3") as vtotal:
        resp = vtotal.request(f"domains/{domain}")
        virus_data = resp.json()['data']['attributes']['last_analysis_stats']
        return virus_data['harmless'], virus_data['malicious'], virus_data['suspicious'], virus_data['undetected'], virus_data['timeout']





In [5]:
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse

#try except outside this func
def link_checker(domain):
    session = HTMLSession()

    r = session.get("https://"+domain)
    unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
    outbound_n = 0
    local_n = 0
    links = []
    for link in unique_netlocs:
        # print(link, unique_netlocs[link])
        links.append(link)
        if domain.lower() in link:
            local_n += unique_netlocs[link]
        else:
            outbound_n += unique_netlocs[link]

    return local_n, outbound_n, local_n + outbound_n, links

# local_n, outbound_links, total_links, links = link_checker('abcnews.com')


In [6]:


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

Options = Options()
Options.headless = True 

domain = 'ralblaw.com'
def check_login(domain, driver):
    wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
    try:
        displayed = driver.find_element('xpath', wp_xpath).is_displayed()

        return displayed
    except:

        return False

def check_page_source(domain, driver):
    try:
        driver.get(f"https://{domain}/wp-admin")
        wp_pg = "wp-content" in driver.page_source
        return wp_pg
    except:

        return False

def check_wp(domain):

    service_obj = Service(executable_path="C:/Users/cvaal/chromedriver.exe")
    driver = webdriver.Chrome(options=Options, service=service_obj)
    driver.implicitly_wait(1)
    driver.get(f"https://{domain}/wp-admin")
    login = check_login(domain, driver)
    pg_source = check_page_source(domain, driver)
    wp = None
    if login == False and pg_source == False:
        wp = False
    else:
        wp = True

    driver.close()
    return wp



   



In [7]:
from urllib.parse import urlparse

with open('FakeNews_URLs.txt', 'r') as f:
    fakenews_sites = f.read().split('\n')
    fakenews_sites = [site.strip() for site in fakenews_sites]

with open('RealNews_URLs.txt', 'r') as f:
    realnews_sites = f.read().split('\n')
    realnews_sites = [site.strip() for site in realnews_sites]
realnews_sites = [urlparse(url).netloc.strip("www.") for url in realnews_sites]

site_dict = {'Fake': fakenews_sites, 'Real': realnews_sites}
site_dict


{'Fake': ['ActionNews3.com',
  'abcnews-us.com',
  '21stCenturyWire.com',
  '100PercentFedUp.com',
  '70news.wordpress.com',
  'Abcnews.com',
  'ActivistPost.com',
  'AmericanNews.com',
  'AnonNews.co',
  'BreakingNews365.net',
  'BreakingNews247.net',
  'BeforeItsNews.com',
  'BigAmericanNews.com',
  'BigPZone.com',
  'ChristWire.org',
  'Chronicle.su',
  'CivicTribune.com',
  'ClickHOle.com',
  'CoastToCoastAM.com',
  'ConsciousLifeNews.com',
  'ConservativeOutfitters.com',
  'ConsipracyWire(WideAwakeAmerica.com)',
  'CounterPsyOps.com',
  'CreamBMP.com',
  'DailyBuzzLive.com',
  'DailyCUrrant.com',
  'DCClothesLine.com',
  'DCGazette.com',
  'DerfMagazine.com',
  'Disclose.tv',
  'DrudgeReport.com.co',
  'DuffleBlog.com',
  'DuhProgressive.com',
  'Embols.com',
  'EmpireNews.com',
  'Endingthefed.com',
  'EnduringVision.com',
  'FPRNradio.com',
  'GeoEngineeringWatch.org',
  'GlobalResearch.ca',
  'GovtSlaves.info',
  'GulagBound.com',
  'HangTheBankers.com',
  'HumansAreFree.com',


In [16]:
from collections import defaultdict

from urllib.request import urlopen, URLError
import socket

def check_url( url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    response = requests.get('https://'+url, timeout=20, headers=headers)
    try:
        return response.status_code < 400 
    except URLError as e:
        return False
    except socket.timeout as e:
        print(False)


In [ ]:
def get_site_data(domain, rating):
    try:
        if check_url(domain) == True:
            pr_int, pr_dec, rank = get_pagerank(domain)
            domain_registrar, postal_code, country = whois_checker(domain)
            harmless, malicious, suspicious, undetected, timeout = virus_scan(domain)
            local_n, outbound_links, total_links, links = link_checker(domain)
            wp_check = check_wp(domain)
            return domain, pr_int, pr_dec, rank, domain_registrar, postal_code, country, harmless, malicious, suspicious, undetected, timeout, local_n, outbound_links, total_links, links, wp_check, rating 
        else:
            return 'Site down'
    except:
        return 'Site down'

In [9]:




    

# domain = 'reddit.com'

columns = ['Domain', 'Page rank integer', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'Harmless', 'Malicious', 'Suspicious', 'Undetected', 'Timeout', 'No. of Local links', 'No. of Outbound links', 'Total links', 'Links to fake news site?', 'Wordpress?', 'Site Rating']

site_df_dict = defaultdict(list)

for rating, domains in site_dict.items():
    for i, domain in enumerate(domains):
        print(f'Analyzing Site {i+1}: {domain} of rating: {rating}')
        site_data = get_site_data(domain.lower(), rating)
        if type(site_data) == tuple:
            for column, data in zip(columns, site_data):
                if type(data) != list:
                    site_df_dict[column].append(data)
                else:
                    if any(site in data for site in fakenews_sites):
                        site_df_dict[column].append(True)
                    else:
                        site_df_dict[column].append(False)
            print('Data Gathered')
        else:
            print('site down')




Analyzing Site 1: ActionNews3.com of rating: Fake
Data Gathered
Analyzing Site 2: abcnews-us.com of rating: Fake
Data Gathered
Analyzing Site 3: 21stCenturyWire.com of rating: Fake
Data Gathered
Analyzing Site 4: 100PercentFedUp.com of rating: Fake
Data Gathered
Analyzing Site 5: 70news.wordpress.com of rating: Fake
site down
Analyzing Site 6: Abcnews.com of rating: Fake
Data Gathered
Analyzing Site 7: ActivistPost.com of rating: Fake
Data Gathered
Analyzing Site 8: AmericanNews.com of rating: Fake
site down
Analyzing Site 9: AnonNews.co of rating: Fake
site down
Analyzing Site 10: BreakingNews365.net of rating: Fake
Data Gathered
Analyzing Site 11: BreakingNews247.net of rating: Fake
Data Gathered
Analyzing Site 12: BeforeItsNews.com of rating: Fake
Data Gathered
Analyzing Site 13: BigAmericanNews.com of rating: Fake
site down
Analyzing Site 14: BigPZone.com of rating: Fake
site down
Analyzing Site 15: ChristWire.org of rating: Fake
site down
Analyzing Site 16: Chronicle.su of rating:

In [10]:
site_df_dict

defaultdict(list,
            {'Domain': ['actionnews3.com',
              'abcnews-us.com',
              '21stcenturywire.com',
              '100percentfedup.com',
              'abcnews.com',
              'activistpost.com',
              'breakingnews365.net',
              'breakingnews247.net',
              'beforeitsnews.com',
              'chronicle.su',
              'civictribune.com',
              'clickhole.com',
              'coasttocoastam.com',
              'consciouslifenews.com',
              'creambmp.com',
              'dailybuzzlive.com',
              'dcclothesline.com',
              'disclose.tv',
              'duffleblog.com',
              'embols.com',
              'endingthefed.com',
              'enduringvision.com',
              'fprnradio.com',
              'geoengineeringwatch.org',
              'globalresearch.ca',
              'humansarefree.com',
              'ifyouonlynews.com',
              'infowars.com',
              'inquisitr.

In [11]:
site_df = pd.DataFrame(site_df_dict)
site_df


In [12]:
# def check_data(domain):
#     pr_int, pr_dec, rank = get_pagerank(domain)
#     domain_registrar, postal_code, country = whois_checker(domain)
#     harmless, malicious, suspicious, undetected, timeout = virus_scan(domain)
#     local_n, outbound_links, total_links, links = link_checker(domain)
#     wp_check = check_wp(domain)
# check_data('21stCenturyWire.com')

In [13]:
# adn.com
# bostonglobe.com
# capitalgazette.com
# charlotteobserver.com
# star-telegram.com
# kansascity.com
# investors.com
# ledger-enquirer.com
# miamiherald.com
# nydailynews.com
# newsobserver.com
# sacbee.com
# seattletimes.com
# sunherald.com
# sj.com 
# washingtonpost.com

In [15]:
with open("unscanned working urls.txt", 'r') as f:
    unscanned_sites = f.read().split('\n')
    unscanned_sites = [site.strip() for site in unscanned_sites]

unscanned_sites

['adn.com',
 'bostonglobe.com',
 'capitalgazette.com',
 'charlotteobserver.com',
 'star-telegram.com',
 'kansascity.com',
 'investors.com',
 'ledger-enquirer.com',
 'miamiherald.com',
 'nydailynews.com',
 'newsobserver.com',
 'sacbee.com',
 'seattletimes.com',
 'sunherald.com',
 'sj.com',
 'washingtonpost.com']

In [19]:

for i, domain in enumerate(unscanned_sites):
    print(f'Analyzing Site {i+1}: {domain} of rating: {rating}')
    site_data = get_site_data(domain.lower(), rating)
    new_row = {}
    if type(site_data) == tuple:
        for column, data in zip(columns, site_data):
            if type(data) != list:
                new_row[column] = data
            else:
                if any(site in data for site in fakenews_sites):
                    new_row[column] = True
                else:
                    new_row[column] = False
        print('Data Gathered')
        site_df.append(new_row, ignore_index=True)

    else:
        print('site down')

Analyzing Site 1: adn.com of rating: Real
site down
Analyzing Site 2: bostonglobe.com of rating: Real
site down
Analyzing Site 3: capitalgazette.com of rating: Real
site down
Analyzing Site 4: charlotteobserver.com of rating: Real
site down
Analyzing Site 5: star-telegram.com of rating: Real
site down
Analyzing Site 6: kansascity.com of rating: Real
site down
Analyzing Site 7: investors.com of rating: Real
site down
Analyzing Site 8: ledger-enquirer.com of rating: Real
site down
Analyzing Site 9: miamiherald.com of rating: Real
site down
Analyzing Site 10: nydailynews.com of rating: Real
site down
Analyzing Site 11: newsobserver.com of rating: Real
site down
Analyzing Site 12: sacbee.com of rating: Real
site down
Analyzing Site 13: seattletimes.com of rating: Real
site down
Analyzing Site 14: sunherald.com of rating: Real
site down
Analyzing Site 15: sj.com of rating: Real
site down
Analyzing Site 16: washingtonpost.com of rating: Real
Data Gathered


In [23]:
site_df[site_df['Domain'] == 'washintonpost.com']

,Domain,Page rank integer,Page rank decimal,Site Rank,Domain registrar,Postal code,Country of origin,Harmless,Malicious,Suspicious,Undetected,Timeout,No. of Local links,No. of Outbound links,Total links,Links to fake news site?,Wordpress?,Site Rating


In [ ]:

site_df.to_csv('FakeReal_News_Sites.csv')

In [18]:
data = {'name': ['Somu', 'Kiku', 'Amol', 'Lini'],
	'physics': [68, 74, 77, 78],
	'chemistry': [84, 56, 73, 69],
	'algebra': [78, 88, 82, 87]}

	
#create dataframe
df_marks = pd.DataFrame(data)
print('Original DataFrame\n------------------')
print(df_marks)

new_row = {'name':['Geo', 'larg'], 'physics':[87, 98], 'chemistry':[82, 92], 'algebra':[75, 97]}
#append row to the dataframe
df_marks = df_marks.append(new_row, ignore_index=True)

print('\n\nNew row added to DataFrame\n--------------------------')
print(df_marks)

Original DataFrame
------------------
   name  physics  chemistry  algebra
0  Somu       68         84       78
1  Kiku       74         56       88
2  Amol       77         73       82
3  Lini       78         69       87


New row added to DataFrame
--------------------------
          name   physics chemistry   algebra
0         Somu        68        84        78
1         Kiku        74        56        88
2         Amol        77        73        82
3         Lini        78        69        87
4  [Geo, larg]  [87, 98]  [82, 92]  [75, 97]
